<p align="right">
<img src="https://raw.githubusercontent.com/anon36424/digital_design_colab/main/Dataflow/media/X.svg"
width="500" height="" style="display: block; margin: 0 auto" />
</p>


<h1>
<summary>
<h1 align="center">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>$\color{midnightBlue}{\underline{\textbf{Dataflow SystemVerilog}}}$<bol><p align="right">
</summary>
</h1>

# Overview
 
## What is a Google Colab Notebook?
 
"Colab notebooks are Jupyter notebooks that run in the cloud and are highly integrated with Google Drive, making them easy to set up, access, and share" - Google

These notebooks make it easy for us to share with you interactive notebooks which are run on virtual machines. This makes it much easier because it doesn't matter what operating system you are running, you can easily download any needed software on the VM.

Another advantage is that you can run the code one block at a time. This allows you to not have to rerun some blocks more than once. 

## What is the point of this notebook?

The goal of this notebook is for you to program and compile a bitstream that you can download onto an FPGA. 

The notebook will explain to you each step from talking about binary to always_comb blocks. 

It uses simple python inputs to check knowledge and more complex verilator scripts to confirm your code does what it is supposed to do. 

## How do I use this notebook?
A notebook consists of a collection of cells.  Some cells display information.  Others contain code.  You interact with cells by "running" them.  For example, the Setup block of code is shown below:

![picture](Dataflow/media/Setup.png)

There are two types of arrows shown in the figure.  The arrow next to the word "Setup" allows you to expand and collapse groups of cells.  Similarly, the arrow next to the phrase "Packages to be downloaded" allows you to reveal the set of packages that will be downloaded on a setup.

However, a different kind of button is shown when you hover over the text "15 cells hidden".  This type of button is a "run button". Clicking that specific button will execute the code to setup the notebook and thus would be the first run button you would click.  When you click a run button you may see a dashed circle around it which slowly fills in to a solid circle - this is the progress bar for that run action.  Some run buttons may take a minute or two to fully run the code they control and so the progress bar will help you see what is happening.  In addition, status messsages at the bottom of your screen show the progress of the running cell.

Only one block of code can run at a time. When a block of code is running, its run button will have a square inside it which can be used to stop it running.  However, stopping blocks of code, especially when downloading things may take some time. 

## How do widgets work?
 
Widgets are interactive modules that enhance the user experience. Press the play button above a widget to load it into the notebook. Then answer the question by changing values or clicking on the boxes. Then hit submit. You can try as many times as you want.

## Does it save my progress?
 
If you make a copy of this notebook, you can save it to your drive. One thing to note, if you stop using it, then you will be disconnected from your Virtual Machine. This means anything downloaded or any files saved will be lost. So if this happens, just rerun setup code blocks and any code blocks that generated files. 
In other words, it will save changes made to the notebook but not any files or changes to the virtual machine.

## Previous Knowledge Required
  * Basic understanding of Binary number system 
  *    Ability to read Truth Tables
  *    Basic knowledge of Programming including variables and branches
  *    Have access to a BASYS 3 board from Xilinx


## **Setup**
Click the Play Button. No need to expand. Do not touch this. 
<details><summary>Packages to be downloaded</summary>
Things it needs to install:  <br>

 *    Verilator- the simulator  <br>
 *    PyGithub - Helps with simulation    <br>
 *    VCD - Generates waveforms from `.vcd` files  <br>
 *    Widgets - Each widget must first be generated then will be loaded when played
</details>

In [ ]:
#@title VCD Install
%%bash
git clone https://github.com/yne/vcd.git
make -C /content/vcd
cd /content/vcd
make install

make: Entering directory '/content/vcd'
cc  vcd.c -o vcd
make: Leaving directory '/content/vcd'
install -v -D vcd /usr/bin/vcd
'vcd' -> '/usr/bin/vcd'


Cloning into 'vcd'...


In [ ]:
#@title Import Files from Repo
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests
def import_text(text):
  url = 'https://raw.githubusercontent.com/anon36424/digital_design_colab/main/bin/%s' % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)

import_text("tb_dataflow_sv.cpp")
import_text("dataflow_sv3.cpp")
import_text("tb_func2.cpp")
import_text("quiz_dict01.py")
import_text("tt_widget.py")
import_text("frquestion.py")

import tt_widget as tt
import quiz_dict01 as quiz_bot 
from frquestion import *
def quiz_func(a):
  quiz_bot.quiz_func(a)

!git clone https://github.com/anon36424/digital_design_colab.git

Cloning into 'FPGA_Colablab'...
remote: Enumerating objects: 977, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 977 (delta 152), reused 221 (delta 123), pack-reused 703
Receiving objects: 100% (977/977), 2.10 MiB | 11.95 MiB/s, done.
Resolving deltas: 100% (466/466), done.


In [ ]:
#@title Anon's Simulation Widget
import os
import re
import sys
import subprocess
from collections import defaultdict, namedtuple
import subprocess
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
!mkdir /content/tmp_code
def vcd():
  !vcd < /content/tmp_code/logs/vlt_dump.vcd > /content/vcd.txt
  !sed -i '1,10d'  /content/vcd.txt 
  !cat /content/vcd.txt


VerilogIO = namedtuple("VerilogIO", "dir nam wid")
'''
        Using a namedtuple (sort of like a C-struct) to hold info on each Verilog IO.
        This is very similar to using a class with only data members.
          The advantages are: (1) the fields are not change-able after creation
          (which is what I want), and (2) probably more efficient than using a class.
        This is used by both generate() and processVmoduleHFile() so it is declared in global scope.
'''


def getModuleName(textSourceCode):
  match_name = re.search(r"module\s{1,}\w{1,}\s{0,}#{0,}\s{0,}\(",textSourceCode.value)
  if match_name is None:
      return None
  mname = match_name.group().split(' ')[1]
  mname = mname.split('(')[0]
  mname = mname.split('#')[0]
  return mname
def writeJson(d, fname='/tmp/json.json', ind=2):
    ''' Write an object to a file with indenting '''
    import json
    with open(fname, encoding='utf-8', mode='w') as f:
        json.dump(d, f, indent=ind)
def cd(folder):
    ''' My wrapper for os.chdir() so I can print it desired. '''
    if folder:
        os.chdir(folder)
    #print("##### Changing to " + os.getcwd() + "#####")

def getNextLine(f):
    ''' Read file f until you get a non-comment and a non-blank link.
        The returned line will be stripped of whitespace (including newline)
    '''
    while True:
        myLine = f.readline().strip()
        if not myLine.startswith('//') and len(myLine) > 0:
            break
    return myLine

def processSTMHeader(tBench):
    '''
        Read STM file and pull out the clock definition and the list of inputs.
        Note that for combinational circuits we are expecting a clock line
          consisting of 'comb' (which is ignored)
    '''
    with open(tBench, encoding='utf-8', mode='r') as f:
        stmClock = None
        tmp = getNextLine(f)
        if tmp != 'comb':
            stmClock = tmp
        inputs = getNextLine(f).split()

    # Verify that a signal doesn't appear twice in the list (this is a user-created file after all)
    for elmt in inputs:
        assert inputs.count(elmt) == 1, "Cannot have duplicate names in STM file input list. ({} appears more than once)".format(elmt)

    return (stmClock, inputs)

def processVmoduleHFile(fname, stmClock, stmInputs):
    '''
        Process the verilator-produced Vmodule.h file and pull out
        the IO definitions.  Pack them into the verilogIOs structure,
        which is a list of lists consisting of [dir, name, size].
        Along the way see if you can match any of the signals as
        corresponding to the clock defined in the stimulus file.
        Finally, make sure that all verilog signals are in the stm file
          and the other way around.
    '''

    verilogIOs = []
    with open(fname, encoding='utf-8', mode='r') as f:
        verilogClock = None
        for line in f:
            line = line.strip()
            if line.startswith('// LOCAL VARIABLES'): # Can skip everything after this line
                break
            match = re.match(r'^VL_(IN|OUT|SIG)(\d*)\(([^,]*),(\d+),(\d+)', line)
            if match is not None:
                varDir, varName, varSize = [match.groups()[0], match.groups()[2], int(match.groups()[3]) - int(match.groups()[4]) +1]
                verilogIOs.append(VerilogIO(varDir, varName, varSize))
                # If input, check that it is in STM file
                if varName == stmClock:
                    verilogClock = varName   # Found matching clock names between STM and Verilog header

    # Check to make sure that every stm input is in the verilog file
    #   and the other way around.
    for si in stmInputs:
        assert si in [f.nam for f in verilogIOs if f.dir=='IN'], \
            "ERROR: input '{}' in STM file but not in Verilog design".format(si)
    for vi in [f.nam for f in verilogIOs if f.dir=='IN']:
        assert vi == stmClock or vi in stmInputs, \
            "ERROR: input '{}' in Verilog design but not in STM file".format(vi)

    # Make sure that if a clock was in the STM file, it was found in the Verilog
    if stmClock != None:
        assert verilogClock != None, "There was a clock defined in the STM file but no corresponding clock in the Verilog file."
    return (verilogClock, verilogIOs)
from string import Template
def generate(designFolder, verilogFile, tBenchFile,
             draw = True, wall = True, compress = True, Hex = True, verbose = False,
             pythonPath = '', processClockWaveform = True,
             hscale = 1, nameOrder = ''):
    """
    Generate a C++ object of verilog/systemverilog code using verilator, toggle inputs on said C++ object,
    and portray resulting waveforms of the signals. Comments in the stm file are supported and are indicated with //
    Comments must start at the beginning of the line and are expected to take up the whole line
    Keyword Arguements:
    designFolder -- str - relative path of the directory, where the .v and .stm files are
    verilogFile -- str - the top level HDL module
    tBenchFile -- str - the name of a .stm file of input stimulus.
    draw -- bool - whether or not to display the waveform
    wall -- bool whether or not to check for verilator warnings
    compress -- bool whether or not to compress the c++ object made by verilator. (Compression effectively
      eliminates the ability to view most internal signals  (deprecated)
    Hex -- bool - whether to display waveform values in hexadecimal
    verbose -- bool - control verbose mode
    pythonPath -- str - path to directory where MaverickSimulator project can be found. If left empty, the typical
                        PYTHONPATH/site-packages will be assumed
    processClockWaveform -- bool -  By default, will attempt to convert the clock waveform to one with arrows.
                                    Setting this to False will prevent this
    hscale -- int - Will adjust the width of the ticks on the waveform diagram. This arguement is passed directly
                    to executeSimulation()
    """
    clock = None    # The name of the clock to the design (if any)
    homeDir = os.getcwd()
    try:
        fileNameRoot = os.path.splitext(verilogFile)[0]
        if not pythonPath:
            dirs = sys.path
            for dir in dirs:
                if (dir.find('site-packages') != -1 ):
                    #TODO add check to make sure path is valid
                    pythonPath = dir + '/MaverickSimulator/src'
                    break
        # Adjust commandline args for verilator
        w = ' -Wall' if wall else ' '
        c = ' ' if compress else ' --public '
        wv = ' --trace ' if draw else ' '

        cd(designFolder)

        # Step 1: run the verilator program
        v = subprocess.Popen(r'verilator -cc'+ w + c + wv + verilogFile + " -CFLAGS '-std=c++11' --exe sim___.cpp",    #create verilator object and link to .cpp
                             stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell = True, universal_newlines=True)
        verbose = 0
        if verbose:
            print('Running verilator...')
            print('verilator -cc'+ w + c + wv + verilogFile + " -CFLAGS '-std=c++11' --exe sim___.cpp")
            output = v.stdout.read().strip()
            if len(output) > 0:
                print('Results of running verilator: ' + output)
            if output.find('%Error: Command Failed') != -1:
                return 1
                sys.exit("Exiting due to verilator errors")
        else:
            output = v.stdout.read().strip()
            if output.find('%Error: Command Failed') != -1:
                print(output)
                os.chdir(homeDir)
                return 1
                sys.exit("Exiting due to verilator errors")

        # Step 2: Parse STM and Verilog files
        # Open .stm file and get clock and list of inputs from the  first two uncommented lines
        stmClock, stmInputs = processSTMHeader(tBenchFile)

        # Read the IO definitions from the Vmodule.h file
        # Add inputs, outputs, and local signals to data structure verilogIOs
        verilogClock, verilogIOs = processVmoduleHFile('obj_dir/V'+fileNameRoot+'.h', stmClock, stmInputs)

        # If verilogClock has been set that means
        # it matched up with the one from the STM file. Otherwise, it did not.
        # The rest of the code will use clock to know this.
        if verilogClock != None:
            clock = verilogClock

        # Step 3:Write the Testbench__lh file
        ios = ''
        sg = ''
        # Build list of IO print stmts for printIOs()
        for nam, wid in [(f.nam, f.wid) for f in verilogIOs if f.dir!='SIG']:
            ios += '    std::cout << " {0} is " << std::bitset<{1}>(top->{0}) <<  std::endl;\n'.format(nam, wid)
        # If needed, build list of IO print stmts for printInternalOs()
        if not compress:
            for nam, wid in [(f.nam, f.wid) for f in verilogIOs if f.dir=='SIG']:
                varName = nam.replace('v__DOT__', '').replace('__DOT__', ' ')
                sg += '   std::cout << "{} is " << std::bitset<{}>(top->{}) << std::endl;\n'.format(varName, wid, nam)
        # Now actually write the testBench file into the design directory
        with open('TestBench___.h', encoding='utf-8', mode='w') as tb: #header file for interacting with verilator object
            with open("/content/tmp_code/TBtemplate.txt", 'r') as tbTemplate:
                t = Template(tbTemplate.read())
                t = t.substitute(
                    MODULE=fileNameRoot,
                    CLKDEF = '\n#define CLK top->'+clock if clock else '',
                    IOPRINTS = ios,
                    INTERNALIOPRINTS = sg
                    )
                tb.write(t)

        # Step 4: write the main simulation file
        # Collect IO print code snippets (using simTemplate2.txt as pattern)
        ios = ''
        for inputName in stmInputs:
            for inputWidth in [f.wid for f in verilogIOs if f.nam==inputName]:
                with open("/content/tmp_code/simTemplate2.txt", 'r') as simTemplate2:
                    t = Template(simTemplate2.read())
                    t = t.substitute(SIGNAME = inputName, SIGWIDTH = inputWidth)
                    ios += t

        # Now write the main simulation file for compilation (using simTemplate.txt as pattern)
        with open('sim___.cpp', encoding='utf-8', mode='w') as sim:
            with open("/content/tmp_code/simTemplate.txt",
                    encoding='utf-8', mode='r') as simTemplate:  #now write main.cpp file
                t = Template(simTemplate.read())
                t = t.substitute(
                            NUMINS = len(stmInputs),
                            READINPUTSCODE = ios,
                            HASCLOCK = (0 if clock is None else 1)
                            )
                sim.write(t)

        # Step 5: Compile the resulting program
        print("Compiling")
        if verbose:
            print('make -C obj_dir -j -f V'+fileNameRoot+'.mk V'+fileNameRoot+'; cd obj_dir')
            v = subprocess.Popen(r'make -C obj_dir -j -f V'+fileNameRoot+'.mk V'+fileNameRoot+'; cd obj_dir',
                                 stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell = True, universal_newlines=True)
            output = v.stdout.read().strip()
            if len(output) > 0:  print('   Results of compiling: ' + output)
        else:
            v = subprocess.Popen(r'make -C obj_dir -j -f V'+fileNameRoot+'.mk V'+fileNameRoot+'; cd obj_dir',    #call makefile to g++ compile appropriate files
                                 stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell = True, universal_newlines=True)

        output = v.stdout.read().strip()
        # Step 6: Execute it
        # if not nameOrder:
        #   names = [f.nam for f in verilogIOs if f.dir != 'SIG']
        #   writeJson(names, fname=fileNameRoot+'.nmo')
        #   nameOrder = fileNameRoot+'.nmo'
        # cd(homeDir)
        # ret = executeSimulation(designFolder, verilogFile, tBenchFile, draw,
        #                     compress, Hex, verbose, clock, nameOrder,
        #                     processClockWaveform, hscale, pythonPath)
        # if draw is False:
        #     return ret
        subprocess.run(["/content/tmp_code/obj_dir/V" +fileNameRoot, "/content/tmp_code" + fileNameRoot + ".stm", "--trace" ])
        # !./tmp_code/obj_dir/VbehavLoadableReg /content/tmp_code/temp.stm --trace
      # !vcd < /content/logs/vlt_dump.vcd
        subprocess.run(["vcd", "<", "/content/logs/vlt_dump.vcd"])
    except Exception as e:
        os.chdir(homeDir)
        print(str(e))
def showWaveformClicked(self):
    sc = cellDict[self]
    assert sc is not None, "Internal error with cellDict"

    if submitCodes(sc.interpreterHomeDir, sc.textSourceCode, sc.textStimulus) == False:
        return
    srcFileName = getModuleName(sc.textSourceCode)
    arrow = sc.chkArrow.value
    hex = True if sc.selectHexBin.value is 'Hex' else False
    assert os.path.exists(sc.interpreterHomeDir+'/tmp_code/'+srcFileName+'.sv'), "Please submit the codes before showing waveform: {} {}".format(sc.interpreterHomeDir, srcFileName)
    generate(sc.interpreterHomeDir+'/tmp_code', srcFileName+'.sv', srcFileName+'.stm', compress=True, Hex=hex, verbose=True, processClockWaveform=arrow)
    if sc.showCode.value:
        print("------------------------------------------------------------------------------------------")
        print("Source code for the simulation above:")
        print("------------------------------------------------------------------------------------------")
        print(sc.textSourceCode.value)
        print("------------------------------------------------------------------------------------------")

    print("###########################################################################################")
    x = subprocess.run(["/content/tmp_code/obj_dir/V" +srcFileName, "/content/tmp_code/" + srcFileName + ".stm", "--trace" ], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    print(x.stderr + x.stdout)
    # x = subprocess.run(["vcd", "< /content/tmp_code/logs/vlt_dump.vcd" ], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    # print(str(x.stderr) + str(x.stdout))
    # # os.system("vcd < /content/tmp_code/logs/vlt_dump.vcd" )
    vcd()
class SimCell:
  def __init__(self, textSourceCode, textStimulus, chkArrow, selectHexBin, interpreterHomeDir, showCode, contents):
      self.textSourceCode = textSourceCode
      self.textStimulus = textStimulus
      self.chkArrow = chkArrow
      self.selectHexBin = selectHexBin
      self.interpreterHomeDir = "/content"
      self.showCode = showCode
      self.contents = contents
def readContents(contents):
    if contents is None:
        code = ''
        stim = ''
    else:
        print(contents)
        contents = "/content/" + contents 
        with open(contents + ".sv") as f:
            code = f.read()
        with open(contents + ".stm") as f:
            stim = f.read()
        # with open(JTEXT+'/pgms/' + contents[:-1] + "tm") as f:
        #     stim = f.read()
    return [code, stim]

cellDict = dict()

def CreateWidgets(contents, ht="125px", wid="500px", stht='125px', stwid='500px'):
    code, stim = readContents(contents)

    srcCodeWidget = widgets.Textarea(
                            disabled=False,
                            value = code,
                            layout=widgets.Layout(height=ht,width=wid),
                            )

    stimulusWidget = widgets.Textarea(
                            value=stim,
                            disabled=False,
                            layout=widgets.Layout(height=stht, width=stwid),
                            )

    selectHexBin = widgets.Dropdown(options=['Binary','Hex'],
                                 value='Hex',
                                 description='',
                                 disabled=False,
                                 )

    chkArrow = widgets.Checkbox(description='Show the clock arrow', value=True, indent=False)
    showCode = widgets.Checkbox(description='Show source code below waveform', value=False, indent=False)
    #butt_submit = widgets.Button(description='Save code')
    butt_show_waveform = widgets.Button(description='Run simulation')
    butt_refresh_contents = widgets.Button(description='Restore code')
    butt_save_code = widgets.Button(description = "Save code")
    return srcCodeWidget, stimulusWidget, selectHexBin, chkArrow, showCode, butt_show_waveform, butt_refresh_contents, butt_save_code
def submitCodes(interpreterHomeDir, textSourceCode, textStimulus):
    if textSourceCode.value is '' or textStimulus.value is '':
        print("The source code and stimulus cannot be empty! Please input again")
        print("###########################################################################################")
        return False
    else:
        with open(interpreterHomeDir + "/tmp_code/" + getModuleName(textSourceCode) + ".sv", 'w') as f:
            f.write(textSourceCode.value)
        with open(interpreterHomeDir + "/tmp_code/" + getModuleName(textSourceCode) + ".stm", 'w') as f:
            f.write(textStimulus.value)
    return True

def refreshContentsClicked(self):
    sc = cellDict[self]
    assert sc is not None, "Internal error with cellDict"
    code, stim = readContents(sc.contents)
    sc.textSourceCode.value = code
    sc.textStimulus.value = stim

def overwriteSavedCode(self):
    sc = cellDict[self]
    print(sc.contents)
    file_location = "/content/" + sc.contents
    with open(file_location + ".sv", "w") as f:
      f.write(sc.textSourceCode.value)
    with open(file_location+".stm", "w") as f:
      f.write(sc.textStimulus.value)

def createSimulationWorkSpace(initialContents = None, ht='125px', wid='500px', stht='125px', stwid='500px'):

    interpreterHomeDir = os.getcwd()
    code_file_path = interpreterHomeDir+'/tmp_code'
    if not os.path.exists(code_file_path):
        os.mkdir(code_file_path)

    # Create widgets for this part
    textSourceCode, textStimulus, selectHexBin, chkArrow, showCode, btnShowWaveform, btnRestoreCode, btnSaveCode = CreateWidgets(initialContents, ht, wid, stht, stwid)
    #print(textSourceCode.value)
    #print(textStimulus.value)

    # Register callbacks for when clicking the button
    btnShowWaveform.on_click(showWaveformClicked)
    btnRestoreCode.on_click(refreshContentsClicked)
    btnSaveCode.on_click(overwriteSavedCode)
    # Register this cell's info
    sc = SimCell(textSourceCode, textStimulus, chkArrow, selectHexBin, interpreterHomeDir, showCode, initialContents)
    cellDict[btnShowWaveform] = sc
    cellDict[btnRestoreCode] = sc
    cellDict[btnSaveCode] = sc


    # Draw cell widgets
    leftPart = widgets.VBox([
                        widgets.VBox([
                                widgets.Label('Source code area:'),
                                textSourceCode
                                ]),
                        widgets.VBox([
                                widgets.Label('Simulation code area:'),
                                textStimulus
                                ])
                        ])
    rightPart = widgets.VBox([
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                #widgets.Label(''),
                                widgets.Label(''),
                                showCode,
                                #widgets.Label(''),
                                btnShowWaveform,
                                #widgets.Label(''),
                                btnSaveCode,
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                btnRestoreCode
                            ])

    # Show widgets
    display(widgets.HBox([leftPart,rightPart]))




In [ ]:
#@title Import Simulation Files
cd("/content/tmp_code")
import_text("TBtemplate.txt")
import_text("simTemplate.txt")
import_text("simTemplate2.txt")
import_text("function1.sv")
import_text("function1.stm")
import_text("function2.sv")
import_text("function2.stm")
import_text("function3.sv")
import_text("function3.stm")
import_text("function4.sv")
import_text("function4.stm")
import_text("dataflow_sv.stm")
import_text("dataflow_sv.sv")
import_text("tb_function1.cpp")
import_text("tb_function2.cpp")
import_text("tb_function3.cpp")
import_text("tb_function4.cpp")

cd("/content")

In [ ]:
#@title Grid Creation
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider

# function of creating button widget
def create_expanded_button(description, button_style, width="100px"):
    return Button(
        description=description,
        button_style=button_style,
        layout=Layout(height="auto", width=width),
    )


# creates variable grid size
def create_grid(num_input, input_string, func):
    """
    - num_input = number of inputs in the function
    - input_string = a string of all the default values and input names that are passed in (ex: AB000011011)
    """
    num_row = (2**num_input) + 1
    num_col = num_input + 2
    grid = GridspecLayout(num_row, num_col, width=str(num_col * 117.5) + "px")

    # creates the default values for the different input combos of the truth table
    for i in range(0, num_row):
        for j in range(0, num_input):
            val = str(input_string[0])
            grid[i, j] = create_expanded_button(val, "info")
            input_string = input_string[1:]

    # creates the click to check button as well as the userinput section of the table
    for i in range(1, num_row):
        grid[i, num_input] = widgets.BoundedIntText(
            min=0, max=1, layout=Layout(height="auto", width="100px")
        )
        grid[0, num_col - 1] = create_expanded_button(
            "Click to Check", "warning", width="150px"
        )

    # creates the output section of the truth table to know if you're right or not
    for i in range(1, num_row):
        grid[i, num_col - 1] = create_expanded_button(" ", "warning", width="150px")

    grid[0, num_input] = create_expanded_button(func, "info")
    return grid


# checks if the user input is correct
def CheckAnswer(grid, num_inputs, input):
    for i in range (0,len(input)):
        if grid[i + 1, num_inputs].value == int(input[i]):
            grid[i + 1, num_inputs + 1].button_style = "Success"
            grid[i + 1, num_inputs + 1].description = "Correct!"
        else:
            grid[i + 1, num_inputs + 1].button_style = "Danger"
            grid[i + 1, num_inputs + 1].description = "Wrong! Submit again"


# AND gate truth table creation
grid1_1 = create_grid(2, 'AB00011011', 'F=AB')

# OR gate truth table creation
grid1_2 = create_grid(2, 'AB00011011', 'F=A+B')

# NOT gate truth table creation
grid1_3 = create_grid(1, "A01", '!A')

# NOR gate truth table creation
grid1_4 = create_grid(2, 'AB00011011', 'F=NOR')

# NAND gate truth table creatione
grid1_5 = create_grid(2, 'AB00011011', 'F=NAND')

# XOR gate truth table creation
grid1_6 = create_grid(2, 'AB00011011', 'F=A^B')

# XNOR gat truth table creatione
grid1_7 = create_grid(2, 'AB00011011', 'F=~(A^B')

# THIS CAN PROBABLY BE PLACED IN THE CREATE FUNCTION WITH F AS AN ARGUMENT ABCF000001010011100101110111


# LOOK INTO HAVING THE CHECK ANSWER BUTTON DO DIFFERENT THINGS
# DON'T LIKE THIS IMPLEMENTATION. Find a way to only need one 'on_button_clicked' function

# Process when clicking the "Check" button
# AND
def on_button_clicked(self):
    CheckAnswer(grid1_1, 2, '0001')


# OR
def on_button_clicked2(self):
    CheckAnswer(grid1_2, 2, '0111')


# NOT
def on_button_clicked3(self):
    CheckAnswer(grid1_3, 1, '10')


# NOR
def on_button_clicked4(self):
    CheckAnswer(grid1_4, 2, '1000')


# NAND
def on_button_clicked5(self):
    CheckAnswer(grid1_5, 2, '1110')


# XOR
def on_button_clicked6(self):
    CheckAnswer(grid1_6, 2, '0110')


# XNOR
def on_button_clicked7(self):
    CheckAnswer(grid1_7, 2, '1001')

In [ ]:
#@title FRQ Creation { form-width: "1000px" }
#TO DO: Refactor this nightmare using a dictionary
#Question 1  
frq_1 = create_frq(frquestions, 0)
def frq_1_check(grid, qlist):
  if grid[0,1].value == qlist[0][1]:
    grid[0,2].button_style = "success"
  else:
    grid[0,2].button_style = "danger"
def check_frq_1(self):
  frq_1_check(frq_1, frquestions)
#Question 2
frq_2 = create_frq(frquestions,1)
def frq_2_check(grid, qlist):
  if grid[0,1].value == qlist[1][1]:
    grid[0,2].button_style = "success"
  else:
    grid[0,2].button_style = "danger"
def check_frq_2(self):
  frq_2_check(frq_2, frquestions)
#Question 3
frq_3 = create_frq(frquestions,2)
def frq_3_check(grid, qlist):
  if grid[0,1].value == qlist[2][1]:
    grid[0,2].button_style = "success"
  else:
    grid[0,2].button_style = "danger"
def check_frq_3(self):
  frq_3_check(frq_3, frquestions)
#Question 4
frq_4 = create_frq(frquestions,3)
def frq_4_check(grid, qlist):
  if grid[0,1].value == qlist[3][1]:
    grid[0,2].button_style = "success"
  else:
    grid[0,2].button_style = "danger"
def check_frq_4(self):
  frq_4_check(frq_4, frquestions)

#Question 5
frq_5 = create_frq(frquestions, 4)
def frq_5_check(grid, qlist):
  if grid[0,1].value == qlist[4][1]:
    grid[0,2].button_style = "success"
  else:
    grid[0,2].button_style = "danger"
def check_frq_5(self):
  frq_5_check(frq_5, frquestions)
frq_5[0,2].on_click(check_frq_5)
#Question 6
frq_6 = create_frq(frquestions, 5)
def frq_6_check(grid, qlist):
  if grid[0,1].value == qlist[5][1]:
    grid[0,2].button_style = "success"
  else:
    grid[0,2].button_style = "danger"
def check_frq_6(self):
  frq_6_check(frq_6, frquestions)
frq_6[0,2].on_click(check_frq_6)
#Question 7
frq_7 = create_frq(frquestions, 6)
def frq_7_check(grid, qlist):
  if grid[0,1].value == qlist[6][1]:
    grid[0,2].button_style = "success"
  else:
    grid[0,2].button_style = "danger"
def check_frq_7(self):
  frq_7_check(frq_7, frquestions)
frq_7[0,2].on_click(check_frq_7)
#Question 8
frq_8 = create_frq(frquestions, 7)
def frq_8_check(grid, qlist):
  if grid[0,1].value == qlist[7][1]:
    grid[0,2].button_style = "success"
  else:
    grid[0,2].button_style = "danger"
def check_frq_8(self):
  frq_8_check(frq_8, frquestions)
frq_8[0,2].on_click(check_frq_8)
#Question 9
frq_9 = create_frq(frquestions, 8)
def frq_9_check(grid, qlist):
  if grid[0,1].value == qlist[8][1]:
    grid[0,2].button_style = "success"
  else:
    grid[0,2].button_style = "danger"
def check_frq_9(self):
  frq_9_check(frq_9, frquestions)
frq_9[0,2].on_click(check_frq_9)
#Question 10
frq_10 = create_frq(frquestions, 9)
def frq_10_check(grid, qlist):
  if grid[0,1].value == qlist[9][1]:
    grid[0,2].button_style = "success"
  else:
    grid[0,2].button_style = "danger"
def check_frq_10(self):
  frq_10_check(frq_10, frquestions)
frq_10[0,2].on_click(check_frq_10)

In [ ]:
#@title Multiple Choice Creation 
from IPython.display import IFrame, display, HTML
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
# function of creating button widget
def create_expanded_button(description, button_style, width='400px'):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width=width))

# function of creating grid layout
def create_grid(A,B,C,D):
    grid = GridspecLayout(5, 3, width='470px')
    grid[0,0]=create_expanded_button(A,'primary')
    grid[1,0]=create_expanded_button(B,'primary')
    grid[2,0]=create_expanded_button(C,'primary')
    grid[3,0]=create_expanded_button(D,'primary')  
    grid[4,0]=create_expanded_button('Submit','Danger')  
    return grid


def check_answermc4(self):
  """
  Used with .on_click to makesure the answer is correct
    self: This is a button. We have added 2 new attributes to it
      current_answer: This is the last clicked button and its description value stored as a str
      answer: This is the correct answer from the mcdict

  """
  if self.current_answer == 0:
    return
  if self.answer[0] == self.current_answer[0]:
    self.button_style = "Success"
    self.description = "Correct!"
  else:
    self.button_style = 'Danger'
    self.description = "Wrong, Try Again!"
"""
key: The question number
pair: This should be a list. indexes 0-3 are the choices. 4 is the answer. 5 is the question
answer: This must be the same str value as the choice
question: This is used in the header
"""
mc_dict = {

    "1": ["A. 0101","B. 01010", "C. 1010","D. Syntax Error", "A", "What is the value of A[5:2] where A = 9’b101010101?"],
    "2": ["A. 11010", "B. 10100", "C. 10111", "D. 01000", "D", "What is the value of A << 3 where A  = 5’b11101 ?"],
    "3": ["A. 10111", "B. 11101", "C. 101", "D. Syntax Error", "A", "What is the Value of {A,B} where A = 3’b101 and B = 2’11?"],
    "4": ["A. 1010", "B. 111000", "C. 101010", "D. Syntax Error", "C", "What is the value of {3{2’b10}}?"],
    "5": ["A.logic input [3:0] x", "B. logic [3:0] input x", "C. input logic x [3:0]", "D. input logic [3:0] x", "D", "Which signal is defined correctly?"  ],
    "6": ["A 1’b1", "B. 1’b0", "C. An Error will be thrown", "D. Undefined", "C", "What is the value of x from the code above?"],
    "7": ["A. 1’b1", "B. 1’b0", "C. 2’b11", "D. An Error will be thrown", "A", "What is the value of x from the above code?"] ,
    "8": [ "A. 4'b0001", "B. 4'b0011", "C. 4'b0111", "D. 4'b1001", "D", "What is the value of x in the above code?" ]
}
def multiple_choice4(key, dictionary):
  """
  Creates a multiple choice widget that has 4 options. 
  key: This is the key to the dictionary. It will be a str of digits. 
  dictionary: the dictionary used is mc_dict. Which houses the multiple choice answers.
  grid: The grid that has been set up and can be shown
  header_button: sets the header_button that houses the question description
  """
  answers = dictionary[key]
  question = answers[5]
  A = answers[0]
  B = answers[1]
  C = answers[2]
  D = answers[3]
  grid = create_grid(A,B,C,D)
  correct_answer = answers[4]
  setattr(grid[4,0], "answer", correct_answer)
  current_answer = 0
  setattr(grid[4,0], "current_answer", 0)
  grid[4,0].button_style = "Warning"
  def change_color(self):
    current_btn = int(self.layout.grid_area[-1])
    for i in range(0,4):
      grid[i,0].button_style = 'primary'
    grid[current_btn-1,0].button_style = 'info'
    grid[4,0].current_answer = self.description

  for i in range(0,4):
    grid[i,0].on_click(change_color)
  grid[4,0].on_click(check_answermc4)
  header_button = create_expanded_button(question, 'info', '465px')
  return grid, header_button



In [ ]:
#@title Verilator Install
!apt-get install verilator >/dev/null
# !pip3 install pyeda > /dev/null
# from pyeda.inter import *

# def create_truth_table(num_inputs, output):
#   X = exprvars('x', num_inputs)
#   f = truthtable(X, output)
#   print(f)

In [ ]:
#@title Install PyGithub
!pip3 install PyGithub > /dev/null

In [ ]:
#@title VCD to Dataframe
#@title vcd2dataframe
def vcd2dataframe(vcd_file):
  with open(vcd_file, "r") as f:
      # Remove Date, Program and  Timescale
      for i in range(20):
          line = f.readline()
          words = line.split()
          try:
            words[0]
          except:
            line = f.readline()
            words = line.split()
            continue
          if words[0] == "$date" or words[0] == "$timescale" or words[0] == "$version":
              for i in range(10):
                  line = f.readline()
                  words = line.split()
                  if words[0] == "$end" or words[-1] == "$end":
                      break
          else:
              break
          line = f.readline()
      # Create the dictionary of lists for inputs
      signals = {}
      line = f.readline()
      words = line.split()
      symbols = {}
      for i in range(50):
          try:
            words[0]
          except:
            line = f.readline()
            words = line.split()
            continue
          if words[0] == "$var":
              signals[words[4]] = []
              symbols[words[3]] = words[4]

          elif words[0] == "$enddefinitions":
              break
          line = f.readline()
          words = line.split()
      # print(signals)
      # print(symbols)
      k = 0

      value = 0
      for i in range(0, 13000):
          if not words:
              line = f.readline()
              words = line.split()
              continue
          elif words[0] == "$dumpvars":
              line = f.readline()
              words = line.split()
              for i in range(50):
                  if words[0] == "$end":
                      break
                  else:
                      lines = f.readline()
                      words = line.split()

          elif len(words) > 1:
              if words[0][0] == "b":
                  # print(line)
                  if words[0][1] == "x":
                      value = 0
                  else:
                      try:
                          value = int(words[0][1:],2)

                      except:
                          value = int(words[0][1],2)
                  symbol = words[1]
                  signals[symbols[symbol]].append(value)
          elif words[0][0] == "#" and words[0][0] != "0":
              k = k + 1
              for key in signals:
                  if len(signals[key]) < k:
                      try:
                          signals[key].append(signals[key][-1])
                      except:
                          signals[key].append(0)
          else:
              if words[0][0] == "x":
                  value = 0
              else:
                  value = int(words[0][0:-1])
              symbol = words[0][-1]
              signals[symbols[words[0][-1]]].append(value)
          line = f.readline()
          words = line.split()
      length_df = 0
      dataframe = signals
      for i in dataframe:
        if length_df < len(dataframe[i]):
          length_df = len(dataframe[i])
        # print(length_df)
      for i in dataframe:
        if len(dataframe[i]) < length_df:
          dataframe[i].append(dataframe[i][-1])
  return dataframe
    # print(signals)
    # Create a list of variables and their names (ignore types)
    # Create a key to translate the signals
    # Begin Cycling through the signals


In [ ]:
# @title Install Interactive Simulation
def altair_sim():
  dataframe = vcd2dataframe("/content/tmp_code/logs/vlt_dump.vcd")
  dataframe1 =dataframe
  df_max = 0
  for i in dataframe:
    if df_max < max(dataframe[i]):
      df_max = max(dataframe[i])
  import altair as alt
  import pandas as pd
  import numpy as np
  from vega_datasets import data
  length_df = 0
  for i in dataframe:
    if len(dataframe[i]) > length_df:
      length_df = len(dataframe[i])
  def make_waveform(dataframe):
    length_df = 0
    for i in dataframe:

      if len(dataframe[i]) > length_df:
        length_df = len(dataframe[i])

    source = pd.DataFrame( dataframe, index=pd.RangeIndex(length_df, name='x'))
    source = source.reset_index().melt('x', var_name='signals', value_name='y')
    scales = alt.selection_interval(bind='scales')

    multi_mouseover = alt.selection_multi(on='mouseover', toggle=True, empty='none')

    selection = alt.selection_multi(fields=['signals'], bind='legend')

    # Create a selection that chooses the nearest point & selects based on x-value
    nearest = alt.selection(type='multi', nearest=True, on='mouseover',
                            fields=['x'], empty='none')

    # The basic line
    line = alt.Chart(source).mark_line(interpolate='step-after').encode(
        x='x:Q',
        y='y:Q',
        color='signals:N'
    )

    # Transparent selectors across the chart. This is what tells us
    # the x-value of the cursor
    selectors = alt.Chart(source).mark_point().encode(
        x='x:Q',
        opacity=alt.value(0),
    ).add_selection(
        nearest,
        scales
    )

    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align='left', dx=5, dy=-5).encode(
        text=alt.condition(nearest, 'y:Q', alt.value(''))
    )

    # Draw a rule at the location of the selection
    rules = alt.Chart(source).mark_rule(color='black').encode(
        x='x:Q',
    ).transform_filter(
        nearest
    )

    # Put the five layers into a chart and bind the data
    waveform=alt.layer(
        line, selectors, points, rules, text
    ).encode(
        alt.X(scale=alt.Scale(domain=(0,length_df))),
        alt.Y(scale=alt.Scale(domain=(0,df_max+1))),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
    ).properties(
        width=600, height=400
    ).add_selection(
        selection
    )

    histogram = alt.Chart(source).mark_bar().encode(
      alt.X('y', 
          scale=alt.Scale(domain=(0,df_max + 1))),
      y='signals',
      color='signals:N',

    ).transform_filter(nearest).properties(
        width = 600, height = 200
    )
    text = histogram.mark_text(
        align = 'left',
        baseline = 'middle',
        dx = 3
    ).encode(
        text = 'y:Q'
    )
    histogram = histogram + text  
    return waveform, histogram

  waveform1, histogram1 = make_waveform(dataframe1)
  return waveform1, histogram1


In [ ]:
#@title vcd2dataframe
#@title vcd2dataframe
def vcd2dataframe(vcd_file):
  with open(vcd_file, "r") as f:
      # Remove Date, Program and  Timescale
      for i in range(20):
          line = f.readline()
          words = line.split()
          try:
            words[0]
          except:
            line = f.readline()
            words = line.split()
            continue
          if words[0] == "$date" or words[0] == "$timescale" or words[0] == "$version":
              for i in range(10):
                  line = f.readline()
                  words = line.split()
                  if words[0] == "$end" or words[-1] == "$end":
                      break
          else:
              break
          line = f.readline()
      # Create the dictionary of lists for inputs
      signals = {}
      line = f.readline()
      words = line.split()
      symbols = {}
      for i in range(50):
          try:
            words[0]
          except:
            line = f.readline()
            words = line.split()
            continue
          if words[0] == "$var":
              signals[words[4]] = []
              symbols[words[3]] = words[4]

          elif words[0] == "$enddefinitions":
              break
          line = f.readline()
          words = line.split()
      # print(signals)
      # print(symbols)
      k = 0

      value = 0
      for i in range(0, 13000):
          if not words:
              line = f.readline()
              words = line.split()
              continue
          elif words[0] == "$dumpvars":
              line = f.readline()
              words = line.split()
              for i in range(50):
                  if words[0] == "$end":
                      break
                  else:
                      lines = f.readline()
                      words = line.split()

          elif len(words) > 1:
              if words[0][0] == "b":
                  # print(line)
                  if words[0][1] == "x":
                      value = 0
                  else:
                      try:
                          value = int(words[0][1:],2)

                      except:
                          value = int(words[0][1],2)
                  symbol = words[1]
                  signals[symbols[symbol]].append(value)
          elif words[0][0] == "#" and words[0][0] != "0":
              k = k + 1
              for key in signals:
                  if len(signals[key]) < k:
                      try:
                          signals[key].append(signals[key][-1])
                      except:
                          signals[key].append(0)
          else:
              if words[0][0] == "x":
                  value = 0
              else:
                  value = int(words[0][0:-1])
              symbol = words[0][-1]
              signals[symbols[words[0][-1]]].append(value)
          line = f.readline()
          words = line.split()
      length_df = 0
      dataframe = signals
      for i in dataframe:
        if length_df < len(dataframe[i]):
          length_df = len(dataframe[i])
        # print(length_df)
      for i in dataframe:
        if len(dataframe[i]) < length_df:
          dataframe[i].append(dataframe[i][-1])
  return dataframe
    # print(signals)
    # Create a list of variables and their names (ignore types)
    # Create a key to translate the signals
    # Begin Cycling through the signals


In [ ]:
#@title Install Extension
!pip install --upgrade git+https://github.com/anon36424/nb_js_diagrammers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/anon36424/nb_js_diagrammers.git to /tmp/pip-req-build-74t7c2ww
  Running command git clone -q https://github.com/anon36424/nb_js_diagrammers.git /tmp/pip-req-build-74t7c2ww
  Created wheel for nb-js-diagrammers: filename=nb_js_diagrammers-0.0.7-py3-none-any.whl size=8911 sha256=bd300f65cecb192a7356a51702082947e2608d0f7f179f8a5d89d65b14f32c19
  Stored in directory: /tmp/pip-ephem-wheel-cache-8tn4id76/wheels/cf/42/33/795922ce5b117354223f1dabb94357b53ce95cfbf41fae959f
Successfully built nb-js-diagrammers


In [ ]:
#@title Load Extension
%load_ext nb_js_diagrammers

In [ ]:
#@title dataframe to wavedrom
def df2wd(filename):
    dataframe = vcd2dataframe("/content/tmp_code/logs/vlt_dump.vcd")
    string_list = []
    name_list = []
    starting_value = 4
    for i in dataframe:
        value = ""
        value_str = ""
        last_value = ""
        for j in dataframe[i]:
            value_list = dataframe[i]
            if last_value == str(j):
                value = "."
            else:
                if int(j) > 2:
                    x = starting_value
                    starting_value += 1
                    if starting_value >= 9:
                        starting_value = 4
                    value = str(x)
                else:
                    value = str(j)
                last_value = str(j)
            value_str = value_str + value
        string_list.append(value_str)
        name_list.append(i)

    wavedrom_format = """{ signal : ["""
    for i in range(len(string_list)):
        if name_list[i] == "clk":
            new_line = (
                """{ name: "clk",  wave: "p"""
                + "." * (-1 + len(value_str))
                + """\", period: 2 },"""
                + f"\n"
            )
        elif max(dataframe[name_list[i]]) > 1:
            int_list = dataframe[name_list[i]]
            new_string_list = string_list[i]
            int_string = ""
            current_value = ""
            last_value = ""
            # Fix String List
            new_string_list =  new_string_list.replace("0", "2")
            new_string_list = new_string_list.replace("1", "3")
            # Create a Data Set
            for j in int_list:
                current_value = j
                if current_value == last_value:
                    pass
                else:
                    int_string += str(current_value) + " "
                last_value = current_value
            new_line = (
                '{ name: "'
                + name_list[i]
                + """\",  wave: \""""
                + new_string_list
                + """\", data:\" """
                + int_string
                + """ \" },"""
                + f"\n"
            )
        else:
            new_line = (
                """{ name: \""""
                + name_list[i]
                + """ \",  wave: \""""
                + string_list[i]
                + """\" },"""
                + f"\n"
            )
        wavedrom_format += new_line
    wavedrom_format += """
    ]}
"""
    filename = "/content/tmp_code/" + filename + ".txt"
    with open(filename, "w") as f:
        f.write(wavedrom_format)



## **Basics of Combinational Logic**


### **Gates: AND, OR and NOT**

***Gates:***
In Combinational Logic a gate is the smallest block. A gate has 2 inputs of binary values, a `1` or `0`, and a single binary output. There are many kinds of gates: AND, OR, NOT, NAND, NOR, XOR, and XNOR. We will go over these gates. 

***Binary:***  
Before we dive into binary, lets analyze our "normal" number system. It is called decimial or base 10. Starting from the right there is a 1's place, then a 10's place, then a 100's place, and so on. These places can also be thought of as 10<sup>0</sup>, 10<sup>1</sup>, and 10<sup>2</sup>. The number at each place represents how many of them there are. So in the number 3765 there are five 1's, six 10's, seven 100's, and three 1000's. So you add all of these up and get the number. 

3\*10<sup>3</sup> + 7\*10<sup>2</sup> + 6\*10<sup>1</sup> + 5\*10<sup>0</sup> = 3765

Binary works exactly the same way but with base 2. You can classify each digit as places, with the 1's place, then the 2's place, then the 4's place and so on. Each place is 2 raised to a higher power. The only values that can be in the places are 1 and 0. Lets follow the same process as above with the binary number 11001.

1\*2<sup>4</sup> + 1\*2<sup>3</sup> + 0\*2<sup>2</sup> + \*2<sup>1</sup> + 1\*2<sup>0</sup> = 16 + 8 + 0 + 0 + 1 = 25

***Overflow:***
Just like in decimal addition, sometimes you need to carry over a number. Like in `9+9` you carry over the one to get `18` because  the result of `9+9` can't be represented by a signal symbol. This of course occurs constantly in binary `11+01 = 100` or `3+1=4` .   

Often in binary we have set number of bits. For example our variable `x` can only be 4 bits wide (or 4 digits wide) because we have 4 wires and each wire is a bit. So if we have `1111` or `15` and we add one it becomes `10000` or `16`. But that is 5 bits wide so the most left bit is *overflow* and gets lost so the actual value is `0000` or `0`.

***Truth Tables***:
When dealing with gates you can draw out a truth table. This represents all of the possible combinations of inputs and their expected outputs. For a single 2-input gate there are 4 possible input combinations: `00`, `01`, `10`, and `11`.

Each digit of the input combination represents a single input wire/signal. Whether that input/signal is high or low is represented by a `1` or a `0`. `1` is high while a `0` is low. In other words, a `1` means that the signal is on, while a `0` means that it is off. 

Each input combination will output either a `1` or `0`. If multiple gates are present, and there are more input wires, the number of possible inputs will increase exponentially.

***NOTE***: In all of these examples the gates have just 2 inputs. But often you may want a gate to have more inputs. Gates can have more thanjust 2 inputs. So the same rules apply to a gate with more than 2 inputs.


#### **AND Gates**:

![picture](https://raw.githubusercontent.com/anon36424/digital_design_colab/main/Dataflow/media/andgate.png)

Just think of the  word AND. If input A AND input B are 1, then the output is also 1. Else, the output is 0. 

In [ ]:
#@title ㅤ
# Events when clicked    
grid1_1[0,3].on_click(on_button_clicked)
# Create the head tab
header_button = create_expanded_button('Figure 1.1: AND Truth Table', 'info', '466px')
# Display the widgets
display(widgets.VBox([header_button,grid1_1]))


#### **OR Gates**:


![picture](https://raw.githubusercontent.com/anon36424/digital_design_colab/main/Dataflow/media/orgate.png)

Just think of the english word OR. If input A OR input B is 1, then the output is also 1. If they are both 1, the output is still 1. If both inputs are 0, then the output is 0. 

In [ ]:
#@title ㅤ
# Events when clicked    
grid1_2[0,3].on_click(on_button_clicked2)
# Create the head tab
header_button = create_expanded_button('Figure 1.2: OR Truth Table', 'info', '466px')
# Display the widgets
display(widgets.VBox([header_button,grid1_2]))


#### **NOT Gates**:

![picture](https://raw.githubusercontent.com/anon36424/digital_design_colab/main/Dataflow/media/notgate.png)

NOT gates are called inverters, the output is the oposite of the input. `1` becomes `0` and `0` becomes `1`.

In [ ]:
#@title ㅤ
# Events when clicked    
grid1_3[0,2].on_click(on_button_clicked3)
# Create the head tab
header_button = create_expanded_button('Figure 1.3: NOT Truth Table', 'info', '350px')
# Display the widgets
display(widgets.VBox([header_button,grid1_3]))


#### **NOR/NAND Gates**:

![picture](https://raw.githubusercontent.com/anon36424/digital_design_colab/main/Dataflow/media/norgate.png)

NOR Gates are OR gates but the output is inverted. The dot is used to represent an invert or NOT gate.

In [ ]:
#@title ㅤ
# Events when clicked    
grid1_4[0,3].on_click(on_button_clicked4)
# Create the head tab
header_button = create_expanded_button('Figure 1.4: NOR Truth Table', 'info', '466px')
# Display the widgets
display(widgets.VBox([header_button,grid1_4]))


![picture](https://raw.githubusercontent.com/anon36424/digital_design_colab/main/Dataflow/media/nandgate.png)

NAND gates are AND gates but the output is inverted

In [ ]:
#@title ㅤ
# Events when clicked    
grid1_5[0,3].on_click(on_button_clicked5)
# Create the head tab
header_button = create_expanded_button('Figure 1.5: NAND Truth Table', 'info', '466px')
# Display the widgets
widgets.VBox([widgets.HTML(value="<span id='1.1'></span>"), AppLayout(header=header_button,footer=None),grid1_5])

#### **XOR/XNOR**

![picture](https://raw.githubusercontent.com/anon36424/digital_design_colab/main/Dataflow/media/xorgate.png)

XOR gates are exclusive OR gates. The output is only 1 if an odd number of the inputs are 1. In this example. if the two inputs are both 0 or they are both 1, the output is 0.

In [ ]:
#@title ㅤ
# Events when clicked    
grid1_6[0,3].on_click(on_button_clicked6)
# Create the head tab
header_button = create_expanded_button('Figure 1.6: XOR Truth Table', 'info', '466px')
# Display the widgets
display(widgets.VBox([header_button,grid1_6]))


![picture](https://raw.githubusercontent.com/anon36424/digital_design_colab/main/Dataflow/media/xnorgate.png)

XNOR gates are XOR gates but the ouput is inverted. The oupt is 1 if an even number of inputs are the same. 

In [ ]:
#@title ㅤ 
# Events when clicked    
grid1_7[0,3].on_click(on_button_clicked7)
# Create the head tab
header_button = create_expanded_button('Figure 1.6: XNOR Truth Table', 'info', '466px')
# Display the widgets
display(widgets.VBox([header_button,grid1_7]))
# display(header_button)
# display(grid1_7)

### **Combinational logic operators**

####***Bitwise***:  



Bitwise Operators: Operators take 2 binary numbers. It then performs an operation on each pair of bits. E.g. It takes the first two bits and does something then the second two bits and so on.

`&` : The equivalent of AND, this operator will AND every bit in the input with the other input starting from left to right and return either a `0` or a `1`. (eg. `1011 & 0110` -> `0010`)

`|` : The equivalent of OR, this operator will OR every bit in the input with the other input starting from left to right and return either a `0` or a `1`. (eg. `1001 | 0011` -> `1011`)

`~` : The equivalent of NOT, this operator will invert every bit of the given input. (eg. `~1011` -> `0100`)

`^` : The equivalent of XOR, this operator will XOR every bit of the inputs and return a `0` is there is an even number of `1`'s or a `1` if there is an odd number of `1`'s. (eg. `1000 ^ 1101` -> `0101`)

In [ ]:
#@title ㅤ
frq_1 = create_frq(frquestions, 0)
frq_1[0,2].on_click(check_frq_1)
header_button = create_expanded_button('Bitwise 1', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_1])

In [ ]:
#@title ㅤ
frq_2 = create_frq(frquestions, 1)
frq_2[0,2].on_click(check_frq_2)
header_button = create_expanded_button('Bitwise 2', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_2])

In [ ]:
#@title ㅤ
frq_3  = create_frq(frquestions, 2)
frq_3[0,2].on_click(check_frq_3)
header_button = create_expanded_button('Bitwise 3', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_3])

####***Reduction***:



Reduction operators convert a string of bits into a single bit. This single bit is the result of performing the operator on every single other bit.
Reductions only take one argument.  
You can think of it as creating a gate with many inputs.

* AND `&`: ANDs every bit (e.g. `&1011`-> `0`, `&1111`-> `1`)
* OR `|`: ORs every bit. (e.g. `|1000` -> `1`)
* XOR `^`: If the number of 1's is odd, its `1` (e.g. `^1101` -> `1`)
* NOT `!`: If the number is `1` or greater it becomes `0`, if it is `0` it becomes `1`. (e.g. `!1011` -> `0`)

In [ ]:
#@title ㅤ
header_button = create_expanded_button('Reduction 1', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_7])

In [ ]:
#@title ㅤ
header_button = create_expanded_button('Reduction 2', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_8])

In [ ]:
#@title ㅤ
header_button = create_expanded_button('Reduction 3', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_9])

In [ ]:
#@title ㅤ
header_button = create_expanded_button('Reduction 4', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_10])

####***Binary:***


This takes two operands and performs the operation on both. It should be used for just single bit wide signals. If it is given a value greater than `1` (`10` OR `111`), it will treat it as `1`. 
* AND `&&`: ANDs the two values (e.g. `101&&100`-> `1`)
* OR `||`: ORs the two values (e.g. `100||000` -> `1`)
* XOR `^^`: XORs the two values

In [ ]:
#@title ㅤ
frq_4 = create_frq(frquestions,3)
frq_4[0,2].on_click(check_frq_4)
header_button = create_expanded_button('Binary 1', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_4])

In [ ]:
#@title ㅤ
header_button = create_expanded_button('Binary 2', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_5])

In [ ]:
#@title ㅤ
header_button = create_expanded_button('Binary 3', 'primary', '550px')
widgets.VBox([AppLayout(header=header_button,footer=None),frq_6])

####***Other Operators:***


**Slice:** 
Constantly, you just want a single bit or a section of bits from a signal you may take a slice of it. `SignalName[X:Y]` where X is the starting bit and Y is the ending bit.
E.g
* A = `4'b0001`, `A[0]` = `1`, `A[1]` = `0`, `A[3:1]` = `000`


In [ ]:
#@title ㅤ
mcgrid1, header_button = multiple_choice4("1", mc_dict)
widgets.VBox([widgets.HTML(value="<span id='something'></span>"), AppLayout(header=header_button,footer=None),mcgrid1])
display(widgets.VBox([header_button,mcgrid1]))



**Bit Shift:**

The `<<` shifts bits to the left, `>>` to the right. The argument on the left is what you want shifted, the argument on the right is how many bits are too be shifted. The new bits are filled with `0`s.  
E.g.   
-   `4'b1010<<2` -> `4'b1000`
- `4'b1111 >> 3` -> `4'b0001`
- `8'b10101010 << 3` -> `8'b01010000`

In [ ]:
#@title ㅤ
mcgrid2, header_button = multiple_choice4("2", mc_dict)
display(widgets.VBox([header_button,mcgrid2]))


**Concatenate:**  
If you want to combine two different signals into one then you use concat `{ }`
This combines the two or signals delimited by a `,`.
* `{2'b10, 2'b01}` -> `1001`





In [ ]:
#@title ㅤ
mcgrid3, header_button = multiple_choice4("3", mc_dict)
display(widgets.VBox([header_button,mcgrid3]))


**Replicate:**  
You can designate for certain signals to be duplicated using the `{X{Y}}` X is the number of times and Y is the signal or value.  
E.g.  
* `{4{1'b1}}` -> `1111` 
* When a = 2'b10: `{{2{2'b10}}, {2{a[0]}}}` -> `101000`

### **Assign and always_comb blocks**


**Defining Signals**

In System Verilog you need to set the size of signals and their type. The most common type is `logic`.   
For example, we need to define an input, with a length of 4 bits. `input logic [3:0] signal_name`

Types of Signals:
`input`: This represents a signal coming from an outside source. For example, a signal from a button, or input from a different module. You cannot directly change an input in a module.
`output`: This represents a signal you want to send to an outside source. For example, to send to an LED or to a different module. You must assign output signals a value. 
`logic`: All signals you will create in this labs will be of the type logic. This is the most standard data type for SystemVerilog. This is not a data type in Verilog.

When using numbers, you need to specify their base, using the format `[width]'[type][value]`
* Binary (Base 2): `b`, width `4`, `4'b10001` = `17`
* Decimal (Base 10): `d`, width `2`, `2'd17` or `17` = `17`
* Octal (Base 8): `o`, width `2`, `2'o21` = `17`
* Hex (Base 16): `h`, width `2`, `2'h11` = `17`

In [ ]:
#@title ㅤ
mcgrid5, header_button = multiple_choice4("5", mc_dict)
display(widgets.VBox([header_button,mcgrid5]))


**RTL**

System Verilog is considered RTL (Register Transfer Logic). Unlike other languages, which are compiled into machine code, System verilog is transformed into physical hardware. This means we must code it a bit differently.

For the most part, System Verilog isn't run line by line, all lines will execute simulatneously. 

There are 3 types of signals, Inputs, Outputs and everything else.   

* Inputs: Usually mapped to a physical component, (e.g. Switches)
* Outputs: Usually mapped to a physical component (e.g. LEDs)
* Signals: All of your other variables

In this lab, your inputs will be the switches, and outputs will be the LEDs. 

**Assign blocks:**

The simplest way to assign variables to values is using an assign statement.     
NOTE: Assigning a multiple values to the same signal will cause an error as all statements are evaluated at the same time. Its like trying to plug in multiple cords to the same USB slot. It just doesn't work.

The key word `assign` is used. The statment afterwards is constantly evaluated.   
Examples `assign a = b & c`;  

An Assign block only works with a single line.

In [ ]:
#@title ㅤ
display(HTML("""
<style>
div.warn {    
    color: #356A89;
    background-color: #D4EAF7;
    border-left: 5px solid #3C82E3;
    padding: 0.5em;
    width: 300px
    }
 </style>
<div class=warn>
 assign x = 1’b1;<br> assign y = 1’b0; <br>assign x = y; </div>"""))

mcgrid6, header_button = multiple_choice4("6", mc_dict)
display(widgets.VBox([header_button,mcgrid6]))


In [ ]:
#@title ㅤ
display(HTML("""
<style>
div.warn {    
    color: #356A89;
    background-color: #D4EAF7;
    border-left: 5px solid #3C82E3;
    padding: 0.5em;
    width: 300px
    }
 </style>
<div class=warn>
 logic x,y; <br> assign x=y;<br> assign y = 2'b11; <br>assign b = 2'b11; </div>"""))

mcgrid7, header_button = multiple_choice4("7", mc_dict)
display(widgets.VBox([header_button,mcgrid7]))


**Always_comb blocks:**

What if we want to use If Else branches or multiple lines without using Assign over and over? We can use an always_comb block.
This turns everything inside into combinational logic.

Always_comb blocks are contained in `begin` and `end` keywords. 
`begin` and `end` keywords are needed when other statements are more than one line long.  

```
always_comb
begin
...
...
end
```

In [ ]:
#@title ㅤ
display(HTML("""
<style>
div.warn {    
    color: #356A89;
    background-color: #D4EAF7;
    border-left: 5px solid #3C82E3;
    padding: 0.5em;
    width: 300px
    }
 </style>
<div class=warn>
logic [1:0] x,y; <br> assign y = 2'b11; <br> always_comb begin<br>if(y==0) x = 1;<br>else if(y==1) x = 4;<br>else if(y==2) x = 7;<br>else if (y==3) x = 9;<br>else x = 0;<br>end
</div>"""))

mcgrid8, header_button = multiple_choice4("8", mc_dict)
display(widgets.VBox([header_button,mcgrid8]))


**If Else Branches:**

We can use if else branches inside an always_comb block.   
NOTE: If you want to have multiple lines after your `if`, `else if` or `else` statement add `begin` and `end`.

These are formed like just like in other programming languages.
```
if (a == 1) begin
...
end
else if (a == 0) begin
...
end
else begin
...
end
```

## **The Lab**

<h3>

<summary><b>Creating a Module</b></summary>
 
</h3>

The fundamental coding blocks of System Verilog are called Modules.   

Think of modules as what you would call functions in other programming languages. You can call other modules and assign different signals to their inputs and outputs.
To Define a module, first use the keyword `module` then give the module a name. Use the names provided in these tutorials. 
Then inside `();` define all of the inputs and outputs separated by a `,`.  

After that, write any code you want in ths module.  
Finally, use the keyword `endmodule` to finish it.   
Example: 
```
module top( 
  input logic a,
  output logic b
  );
assign b = a;
endmodule
```

<h3>

<summary><b>Testing with a Simulator</b></summary>
 
</h3>

How the Simulator works:  

To verify System Verilog code testbenches and simulators are used. Large and even small SystemVerilog projects can take a long time to go through the tool chain, but simulation is quick. So it is beneficial to make sure no bugs or errors exist in the code before compiling.

We will be using the Verilator simulator. It converts System Verilog code into C++. Then uses a C++ wrapper to define inputs and checks the output. 

Using the widget below you will be able to write and simulate System Verilog. 
Functions:
*   __Run Simulation:__ Converts the Verilog and simulation code into C++ to be run with verilator. Then converts it to a `.vcd` file which is used to generate the waveform.
*   __Save Code:__ Overwrites the files saved on colabs with whatever is in your two boxes. There is a `.sv` and a `.stm` file.
*   __Restore Code:__ Loads the saved files back into the box. If you used Save Code, that is the code that will be restored.

NOTE: Make sure you correctly name all signals or errors will occur with the testbenches.

<h3>

<summary><b>Overview of the Project</b></summary>
 
</h3>
Because of how testbenches and Verilator works, its important that you define all of your signals with the provided names and you define you modules with the provided names. If you used a different name you can change the name in Verilator but be careful.
 
You will implement 4 different functions. Each function will only be active when its button is pressed. If no button is pressed, the LEDs should show which switches are flipped.

Each function will take the input (switches) and change it. This will be displayed on the LEDs.


#### ***Function 1:***


Down Button (btnd): Shift all the bits of the input to the left 3 times when you press the down button.  
If the button is not pressed, the LEDs should show the value of the switches.  
Input your code below for this function. When ready run it and it will be linted then tested. If it doesn't work, make changes and try again.

In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/function1")

tmp_code/function1


tmp_code/function1
Compiling
###########################################################################################
b'Enabling waves into logs/vlt_dump.vcd...\n'
  function1:
    led [15:0] : "0    B    0    8    FA50 18   38   78   "
    btnd       : "▁▁▁▁▁▁▁▁▁▁╱▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔"
    sw [15:0]  : "0    B    0    1    1F4A 3    7    F    "


In [ ]:
#@title Note
display(HTML("""
<style>
div.warn {    
    color: #356A89;
    background-color: #D4EAF7;
    border-left: 5px solid #3C82E3;
    padding: 0.5em;
    width: 900px
    }
 </style>
<div class=warn>
NOTE: Simulators will show a waveform based on your code. They use a .vcd file which is generated when you hit "Run simulation". Everytime you simulte you overwrite the last .vcd file. If you forget to simulate again, the wave form viewers will show the last .vcd file you had. 
</div>"""))

In [ ]:
#@title Create WaveDrom
df2wd("function1")

In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/function1.txt
---

In [ ]:
#@title Verilator TestBench
!verilator --cc function1.sv 
!verilator -Wall --trace -cc function1.sv --exe tb_function1.cpp
!make -C obj_dir -f Vfunction1.mk Vfunction1 > /dev/null
!./obj_dir/Vfunction1

CORRECT: led = 0 while sw = 0 and btnd = 0
CORRECT: led = 11 while sw = 11 and btnd = 0
CORRECT: led = 0 while sw = 0 and btnd = 1
CORRECT: led = 64080 while sw = 8010 and btnd = 1
CORRECT: led = 24 while sw = 3 and btnd = 1


In [ ]:
#@title Interactive Simulator
waveform1, histogram1 = altair_sim()
waveform1 & histogram1

alt.VConcatChart(...)

#### ***Function 2:***


Left Button (btnl): Using all 16 switches and LEDs, do a bitwise XOR with the left and right half of the input. Set the left half of the output to 0, and the right half as the result of the XOR  

If the button is not pressed, the LEDs should show the value of the switches.


In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/function2")

tmp_code/function2


Compiling
###########################################################################################
b'Enabling waves into logs/vlt_dump.vcd...\n'
  function2:
    sw [15:0]  : "0    4D2  221E ABAB 2F42 1DDA FFFF FFFF FFFF "
    btnl       : "▁▁▁▁▁▁▁▁▁▁╱▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔╲▁▁▁▁╱▔▔▔▔"
    led [15:0] : "0    4D2  3C   0    6D   C7   0    FFFF 0    "


In [ ]:
#@title Create WaveDrom
df2wd("function2")

In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/function2.txt
---

In [ ]:
#@title Verilator TestBench
!verilator -Wall -cc function2.sv --exe tb_function2.cpp
!make -C obj_dir -f Vfunction2.mk Vfunction2 > /dev/null
!./obj_dir/Vfunction2

CORRECT: led = 0 while sw = 0 and btnl = 0
CORRECT: led = 1234 while sw = 1234 and btnl = 0
CORRECT: led = 60 while sw = 8734 and btnl = 1
CORRECT: led = 109 while sw = 12098 and btnl = 1


In [ ]:
#@title Interactive Simulator
waveform1, histogram1 = altair_sim()
waveform1 & histogram1

alt.VConcatChart(...)

#### ***Function 3:***


Down button (btnr): Implement the following function while btnr is high:  
![picture](https://raw.githubusercontent.com/anon36424/digital_design_colab/main/Dataflow/media/function3.png)
  
If btnr is low then have the led be equal to sw.

NOTE: A will be the first switch, B the second and C the third. F will be the first LED

Hint: `A = sw[0]`. You may also create intermediate signals if you prefer
  

If the button is not pressed, the LEDs should show the value of the switches.


In [ ]:
#@title Launch Simulation Workspace

createSimulationWorkSpace("tmp_code/function3")

tmp_code/function3


Compiling
###########################################################################################
b'Enabling waves into logs/vlt_dump.vcd...\n'
  function3:
    sw [12:0]  : "0    1FFF 1FFF 1FFF 1FFF 1FFF 1FFF 1FFF 1FFF 1FFF "
    btnr       : "▁▁▁▁▁╱▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔"
    A          : "▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁╱▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔"
    B          : "▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁╱▔▔▔▔▔▔▔▔▔╲▁▁▁▁▁▁▁▁▁╱▔▔▔▔▔▔▔▔▔▔▔▔▔▔"
    C          : "▁▁▁▁▁▁▁▁▁▁╱▔▔▔▔╲▁▁▁▁╱▔▔▔▔╲▁▁▁▁╱▔▔▔▔╲▁▁▁▁╱▔▔▔▔▔▔▔▔▔"
    led [15:0] : "0    0    1    0    1    1    1    0    1    1    "


In [ ]:
#@title Create WaveDrom
df2wd("function3")


In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/function3.txt
---

In [ ]:
#@title Interactive Simulator
waveform1, histogram1 = altair_sim()
waveform1 & histogram1

alt.VConcatChart(...)

#### ***Function 4:***


Up Button btnu:  
The door to the digital design lab has an electronic lock. When someone swipes their ID card, it could be a student or a professor. If it is a student, and it is between 7am and 10pm, the door will unlock. If it is a professor, the door will open no matter what time it is. Implement this functionality when the up button is pressed.

**Inputs:**  
Sw[0] - is student?  
Sw[1] - is professor?  
Sw[2] - 0 = am, 1 = pm  
Sw[6:3] - binary representation of the hour of the day  
**Output:**  
If the door unlocks, turn on all LEDs

**Note:** If the studnet input and the professor input are both high, the door should stay locked (this should not be possible—it is an imposter). If you recieve an invalid time (ex: 15:00am) the door should also stay locked.  


If the button is not pressed, the LEDs should show the value of the switches.


In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/function4")

tmp_code/function4


Compiling
###########################################################################################
b'Enabling waves into logs/vlt_dump.vcd...\n'
  function4:
    btnu       : "▁▁▁▁▁▁▁▁▁▁╱▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔"
    student    : "▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁╱▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔╲▁▁▁▁╱▔▔▔▔"
    prof       : "▁▁▁▁▁╱▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔▔╲▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁╱▔▔▔▔╲▁▁▁▁╱▔▔▔▔"
    pm         : "▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁╱▔▔▔▔╲▁▁▁▁╱▔▔▔▔╲▁▁▁▁╱▔▔▔▔╲▁▁▁▁╱▔▔▔▔╲▁▁▁▁▁▁▁▁▁╱▔▔▔▔"
    hour [3:0] : "0    B    B    B    2    2    B    B    2    2    B    B    1    "
    sw [8:0]   : "1FF  1FF  1FF  1FF  1FF  1FF  1FF  1FF  1FF  1FF  1FF  1FF  1FF  "
    led [15:0] : "FF80 FFDA FFFF FFFF FFFF FFFF FFFF 0    0    FFFF 0    0    0    "


In [ ]:
#@title Create WaveDrom
df2wd("function4")


In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/function4.txt
---

In [ ]:
#@title Verilator TestBench
!verilator -Wall -cc function4.sv --exe tb_function4.cpp
!make -C obj_dir -f Vfunction4.mk Vfunction4 > /dev/null
!./obj_dir/Vfunction4

../tb_function4.cpp: In function ‘int main(int, char**, char**)’:
../tb_function4.cpp:152:10: error: ‘class Vfunction4’ has no member named ‘trace’
     dut->trace(m_trace, 5);
          ^~~~~
make: *** [tb_function4.o] Error 1
Please provide input file


In [ ]:
#@title Interactive Simulator
waveform1, histogram1 = altair_sim()
waveform1 & histogram1

alt.VConcatChart(...)

#### ***Bringing it together***





Now add all of your functions together. When a certain button is pressed it should implement that function. When no button is pressed, it the leds should show the value of the switches.

You will just combine all of your functions together.

The module should be called `dataflow_sv`
It should have `sw` as a 16 bit wide inputs.
It should have `btnl`, `btnd`, `btnr`, and `btnu` as single bit inputs.
It should have `led` as a 16 bit wide output

You will have to rename and refactor some of your inputs. For example, on Function 3, instead of having A, B, and C be inputs, you will need to assign them as intermediate signals:
```
assign A = sw[2];
assign B = sw[1];
assign C = sw[0];
```

This is because one input is mapped to one signal.

You should use if/else if/else branches to switch between your 4 functions.


| Module Name: |dataflow_sv|||
| ----------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|btnd 	|Input 	|1| Activates Function1	|
|btnl 	|Input 	|1| Activates Function2	|
|btnr 	|Input 	|1| Activates Function3	|
|btnu 	|Input 	|1| Activates Function4	|
|sw 	|Input 	|16 |	The value of the switches|
|led| 	Output 	|16| The value to display on the LEDs|

In [ ]:
#@title Hint
%%html

<style>
div.warn {    
    color: #356A89;
    background-color: #D4EAF7;
    border-left: 5px solid #3C82E3;
    padding: 0.5em;
    }
 </style>
<div class=warn>
The module should be called dataflow_sv. It should have sw as a 16 bit wide inputs. <br> 
It should have btnl, btnd, btnr, and btnu as single bit inputs. It should have led as a 16 bit wide output.
</div>


In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/dataflow_sv")

tmp_code/dataflow_sv


tmp_code/dataflow_sv
Compiling
###########################################################################################
b'Enabling waves into logs/vlt_dump.vcd...\n'
  dataflow_sv:
    sw [15:0]  : "7B   7B   FD   FD   "
    btnu       : "▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁"
    btnd       : "▁▁▁▁▁╱▔▔▔▔▔▔▔▔▔╲▁▁▁▁"
    btnl       : "▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁"
    btnr       : "▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁"
    led [15:0] : "7B   3D8  7E8  FD   "
tmp_code/dataflow_sv


#### ***XDC Files:***




XDC stands for Xilinx Design Constraints. The master XDC file  has each of the ports for input and output on the Basys3 board. The XDC file connects signals with physical hardware. This will be needed to bind signals to switches, LEDs and buttons. 

Uncomment the lines containing `btnr`, `btnl`, `btnu`, and `btnd`. This tells the FPGA what input is linked to button.

***NOTE***: In future labs, if you're clever about how you name your signals in your top level module, you won't need to rename any signals in the .xdc but will just uncomment the lines that you need.

You will need to uncomment the lines that contain the button signals.
The lines look like this:  
`# set_property -dict { PACKAGE_PIN U18   IOSTANDARD LVCMOS33 } [get_ports { btnc }];`

Then run the codeblock to save your constraints file.

In [ ]:
#@title Generate an XDC File
%%bash -c 'cat > /content/tmp_code/xdc.xdc'

## Buttons
# set_property -dict { PACKAGE_PIN U18   IOSTANDARD LVCMOS33 } [get_ports { btnc }];
set_property -dict { PACKAGE_PIN T18   IOSTANDARD LVCMOS33 } [get_ports { btnu }];
set_property -dict { PACKAGE_PIN W19   IOSTANDARD LVCMOS33 } [get_ports { btnl }];
set_property -dict { PACKAGE_PIN T17   IOSTANDARD LVCMOS33 } [get_ports { btnr }];
set_property -dict { PACKAGE_PIN U17   IOSTANDARD LVCMOS33 } [get_ports { btnd }];

## Switches
set_property -dict { PACKAGE_PIN V17   IOSTANDARD LVCMOS33 } [get_ports { sw[0] }];
set_property -dict { PACKAGE_PIN V16   IOSTANDARD LVCMOS33 } [get_ports { sw[1] }];
set_property -dict { PACKAGE_PIN W16   IOSTANDARD LVCMOS33 } [get_ports { sw[2] }];
set_property -dict { PACKAGE_PIN W17   IOSTANDARD LVCMOS33 } [get_ports { sw[3] }];
set_property -dict { PACKAGE_PIN W15   IOSTANDARD LVCMOS33 } [get_ports { sw[4] }];
set_property -dict { PACKAGE_PIN V15   IOSTANDARD LVCMOS33 } [get_ports { sw[5] }];
set_property -dict { PACKAGE_PIN W14   IOSTANDARD LVCMOS33 } [get_ports { sw[6] }];
set_property -dict { PACKAGE_PIN W13   IOSTANDARD LVCMOS33 } [get_ports { sw[7] }];
set_property -dict { PACKAGE_PIN V2    IOSTANDARD LVCMOS33 } [get_ports { sw[8] }];
set_property -dict { PACKAGE_PIN T3    IOSTANDARD LVCMOS33 } [get_ports { sw[9] }];
set_property -dict { PACKAGE_PIN T2    IOSTANDARD LVCMOS33 } [get_ports { sw[10] }];
set_property -dict { PACKAGE_PIN R3    IOSTANDARD LVCMOS33 } [get_ports { sw[11] }];
set_property -dict { PACKAGE_PIN W2    IOSTANDARD LVCMOS33 } [get_ports { sw[12] }];
set_property -dict { PACKAGE_PIN U1    IOSTANDARD LVCMOS33 } [get_ports { sw[13] }];
set_property -dict { PACKAGE_PIN T1    IOSTANDARD LVCMOS33 } [get_ports { sw[14] }];
set_property -dict { PACKAGE_PIN R2    IOSTANDARD LVCMOS33 } [get_ports { sw[15] }];

## LEDs
set_property -dict { PACKAGE_PIN U16   IOSTANDARD LVCMOS33 } [get_ports { led[0] }];
set_property -dict { PACKAGE_PIN E19   IOSTANDARD LVCMOS33 } [get_ports { led[1] }];
set_property -dict { PACKAGE_PIN U19   IOSTANDARD LVCMOS33 } [get_ports { led[2] }];
set_property -dict { PACKAGE_PIN V19   IOSTANDARD LVCMOS33 } [get_ports { led[3] }];
set_property -dict { PACKAGE_PIN W18   IOSTANDARD LVCMOS33 } [get_ports { led[4] }];
set_property -dict { PACKAGE_PIN U15   IOSTANDARD LVCMOS33 } [get_ports { led[5] }];
set_property -dict { PACKAGE_PIN U14   IOSTANDARD LVCMOS33 } [get_ports { led[6] }];
set_property -dict { PACKAGE_PIN V14   IOSTANDARD LVCMOS33 } [get_ports { led[7] }];
set_property -dict { PACKAGE_PIN V13   IOSTANDARD LVCMOS33 } [get_ports { led[8] }];
set_property -dict { PACKAGE_PIN V3    IOSTANDARD LVCMOS33 } [get_ports { led[9] }];
set_property -dict { PACKAGE_PIN W3    IOSTANDARD LVCMOS33 } [get_ports { led[10] }];
set_property -dict { PACKAGE_PIN U3    IOSTANDARD LVCMOS33 } [get_ports { led[11] }];
set_property -dict { PACKAGE_PIN P3    IOSTANDARD LVCMOS33 } [get_ports { led[12] }];
set_property -dict { PACKAGE_PIN N3    IOSTANDARD LVCMOS33 } [get_ports { led[13] }];
set_property -dict { PACKAGE_PIN P1    IOSTANDARD LVCMOS33 } [get_ports { led[14] }];
set_property -dict { PACKAGE_PIN L1    IOSTANDARD LVCMOS33 } [get_ports { led[15] }];



## **Compiling with the F4PGA Toolchain**


### Installing the Toolchain

Click play. This may take up to 4 minutes.

In [ ]:
!apt install -y git wget xz-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.13).
wget is already the newest version (1.19.4-1ubuntu2.2).
xz-utils is already the newest version (5.2.2-1.3ubuntu0.1).
xz-utils set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
%%bash
git clone https://github.com/chipsalliance/f4pga-examples

Cloning into 'f4pga-examples'...


In [ ]:
%%bash
cd f4pga-examples
time wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O conda_installer.sh

--2022-11-12 22:03:25--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-11-12 22:03:25--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘conda_installer.sh’

     0K .......... .......... .......... .......... ..........  0% 9.76M 7s
    50K .......... .......... .......... .......... ..........  0% 11.1M 7s
 

In [ ]:
#Creates the Conda Environment
%%bash
cd f4pga-examples
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
time bash conda_installer.sh -u -b -p $F4PGA_INSTALL_DIR/$FPGA_FAM/conda;
time source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
time conda env create -f $FPGA_FAM/environment.yml

PREFIX=/root/opt/f4pga/xc7/conda
Unpacking payload ...
Solving environment: ...working... done

## Package Plan ##

  environment location: /root/opt/f4pga/xc7/conda

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py39h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py39h06a4308_2
    - cffi==1.15.0=py39hd667e15_1
    - charset-normalizer==2.0.4=pyhd3eb1b0_0
    - colorama==0.4.4=pyhd3eb1b0_0
    - conda-content-trust==0.1.1=pyhd3eb1b0_0
    - conda-package-handling==1.8.1=py39h7f8727e_0
    - conda==4.12.0=py39h06a4308_0
    - cryptography==36.0.0=py39h9ce1e76_0
    - idna==3.3=pyhd3eb1b0_0
    - ld_impl_linux-64==2.35.1=h7274673_9
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.3.0=h5101ec6_17
    - libgomp==9.3.0=h5101ec6_17
    - libstdcxx-ng==9.3.0=hd4cf53a_17
    - ncurses==6.3=h7f8727e_2
    - openssl==1.1.1n=h7f8727e_0
    - pip==21.2.4=py39h06a4308_0
    - pycosat==0.6.3=py39h27cfd23

                                                                                             
real	0m11.775s
user	0m9.423s
sys	0m3.165s

real	0m0.006s
user	0m0.003s
sys	0m0.004s


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



real	2m31.760s
user	1m37.980s
sys	0m13.467s


In [ ]:
#Download F4PGA Arch Defs
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda activate xc7
mkdir -p $F4PGA_INSTALL_DIR/xc7/install

F4PGA_TIMESTAMP='20220920-124259'
F4PGA_HASH='007d1c1'
export F4PGA_PACKAGES='install-xc7 xc7a50t_test'

for PKG in $F4PGA_PACKAGES; do
  wget -qO- https://storage.googleapis.com/symbiflow-arch-defs/artifacts/prod/foss-fpga-tools/symbiflow-arch-defs/continuous/install/${F4PGA_TIMESTAMP}/symbiflow-arch-defs-${PKG}-${F4PGA_HASH}.tar.xz | tar -xJC $F4PGA_INSTALL_DIR/${FPGA_FAM}
done

### Compiling the Lab with the Toolchain
This should take up to 3 minutes

In [ ]:
#This creates the Makefile
with open("/content/tmp_code/Makefile", "w") as f:
  f.write("""current_dir := ${CURDIR}
TARGET := basys3

TOP := dataflow_sv

XDC := ${current_dir}/*.xdc

SOURCES := ${current_dir}/dataflow_sv.sv

include /content/tmp_code/f4pga-examples/common/common.mk
""")

In [ ]:
#Make the project
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
export TARGET=basys3
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
mkdir /content/Dataflow_SV
cd /content/tmp_code
mv xdc.xdc /content/Dataflow_SV/xdc.xdc
mv Makefile /content/Dataflow_SV/Makefile
mv dataflow_sv.sv /content/Dataflow_SV/dataflow_sv.sv
conda activate xc7
cd /content/Dataflow_SV
time make
# make clean
# make #2> ./error.log 1>./compile.log
cp /content/Dataflow_SV/build/basys3/*.bit /content/dataflow_sv.bit

mkdir -p /content/Dataflow_SV/build/basys3
cd /content/Dataflow_SV/build/basys3 && symbiflow_synth -t dataflow_sv  -v /content/Dataflow_SV/dataflow_sv.sv -d artix7 -p xc7a35tcpg236-1 -x /content/Dataflow_SV/*.xdc
[F4PGA] Running (deprecated) synth
-t
dataflow_sv
-v
/content/Dataflow_SV/dataflow_sv.sv
-d
artix7
-p
xc7a35tcpg236-1
-x
/content/Dataflow_SV/xdc.xdc

 /----------------------------------------------------------------------------\
 |                                                                            |
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |                                                                            |
 |  Copyright (C) 2012 - 2020  Claire Xenia Wolf <claire@yosyshq.com>         |
 |                                                                            |
 |  Permission to use, copy, modify, and/or distribute this software for any  |
 |  purpose with or without fee is hereby granted, provided that the above  

tcmalloc: large alloc 1342177280 bytes == 0xb4bea000 @  0x7f9a867e0887 0x552e36 0x4c3024 0x4ba96b 0x46b9fc 0x46bc38 0x5eb5d7 0x7f9a85e29c87 0x5dbb1a

real	2m17.866s
user	1m14.874s
sys	1m1.291s


## **Testing it on the board**

To Download it to the board you will need OpenOCD.  

First create a folder to house the files on your local machine. Then download the OpenOCD configure file and the bitstream from this Notebook and add then to your folder (they will be in a zip file in /content/dataflow_sv.zip). On command line go into the folder and run the command   

`openocd --file dataflow_sv.cfg`

It should only take a few moments to download to the board.

In [ ]:
#@title Create File for openocd
with open("/content/dataflow_sv.cfg", "w") as f:
  f.write("""interface ftdi
ftdi_device_desc "Digilent USB Device"
ftdi_vid_pid 0x0403 0x6010
# channel 1 does not have any functionality
ftdi_channel 0
# just TCK TDI TDO TMS, no reset
ftdi_layout_init 0x0088 0x008b
reset_config none
adapter_khz 10000

source [find cpld/xilinx-xc7.cfg]
source [find cpld/jtagspi.cfg]
init

puts [irscan xc7.tap 0x09]

set xc7a35t "0362D093"
set xc7a100t "13631093"
set code [drscan xc7.tap 32 0]  
puts $code

if { $code == $xc7a35t} {
    puts "The board has an xc7a35t"
}

if { $code == $xc7a100t} {
    puts "The board has an xc7a100t"
}

puts "Programming..."
pld load 0 dataflow_sv.bit
exit"""
)

In [ ]:
#@title Create Zip
%cd /content
!mv Dataflow_SV/build/basys3/*.bit ./
!zip dataflow_sv.zip *.bit dataflow_sv.cfg

/content
mv: cannot stat 'Dataflow_SV/build/basys3/*.bit': No such file or directory
updating: dataflow_sv.bit (deflated 100%)
updating: dataflow_sv.cfg (deflated 41%)


NameError: ignored